In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd

import devkit.api as dk
import finkit.api as fk
import DataAPI

Populating the interactive namespace from numpy and matplotlib


---

# 数据库架构

In [3]:
db_names = [
    "index_contents",  # 股票列表
    "industry",  # 行业分类
    "indicator",  # 基础指标
    "factor",  # 因子
    "factor_return",  # 因子收益
]

# db_name = "INDICATOR"
for db_name in db_names:
    DataAPI.schema.show_db_info(db_name)

数据库 [index_contents] 架构信息：

子数据库 [000016.SH]
    说明：上证50成分股编号、简称和权重信息
    开始时间：2016-01-04
    结束时间：2018-03-02
    最后更新：2018-03-07 20:36:49

子数据库 [000300.SH]
    说明：沪深300成分股编号、简称和权重信息
    开始时间：2006-01-25
    结束时间：2018-05-31
    最后更新：2018-06-26 21:22:59

子数据库 [000905.SH]
    说明：中证500成分股编号、简称和权重信息
    开始时间：2016-01-04
    结束时间：2018-03-02
    最后更新：2018-03-07 20:36:50

子数据库 [A]
    说明：全A股编号和简称信息
    开始时间：2005-01-04
    结束时间：2018-12-31
    最后更新：2018-06-28 12:40:07

子数据库 [H]
    说明：全H股编号和简称信息
    开始时间：2017-01-03
    结束时间：2018-01-04
    最后更新：2018-03-07 13:50:49

子数据库 [HSI.HI]
    说明：恒生指数成分股编号和简称信息
    开始时间：2016-01-04
    结束时间：2018-01-30
    最后更新：2018-01-30 19:58:42

数据库 [industry] 架构信息：

子数据库 [A_SWL1]
    说明：全A股申万一级行业分类
    开始时间：2015-01-05
    结束时间：2018-01-04
    最后更新：2018-03-07 13:55:33

子数据库 [H_GICSL1]
    说明：全H股GICS一级行业分类
    开始时间：2018-01-02
    结束时间：2018-01-03
    最后更新：2018-03-07 13:55:53

子数据库 [H_SWL1]
    说明：全H股申万一级行业分类
    开始时间：2017-12-29
    结束时间：2017-12-29
    最后更新：2018-03-12 10:59:18


---

# 待更新日期

In [2]:
start = "2005-04-01"
end = "2018-06-31"
trading_days = fk.get_monthly_last_trading_days(start=start, end=end)
# trading_days = fk.get_monthly_last_trading_days(start=start, end=end)
#trading_days = list(set(fk.get_trading_days(start=start, end=end)).union(set(fk.get_report_days(start=start, end=end))))
trading_days = sorted(trading_days)

In [3]:
a = pd.DataFrame(DataAPI.schema.get_schema("indicator")).T
inds_ts = a[a.type == "时间序列"].index.tolist()
inds_fr = a[a.type == "财报数据"].index.tolist()

---

# indicator

In [3]:
# indicators = inds_ts  # 为空表示更新所有存在的indicator
DataAPI.write.update_indicators(["HIGH", "LOW", "IPO_LISTDAYS"], trading_days, override=False, log=True)

[INFO] 2018-07-10 23:40:19 Updating indicator HIGH
[INFO] 2018-07-10 23:40:19 E:\02_work\sw\db\indicator\2005.db is connected.
CREATE TABLE [HIGH] (
    date CHAR(10) NOT NULL,
    sec_id TEXT NOT NULL,
    HIGH REAL
);
[INFO] 2018-07-10 23:40:19 TABLE HIGH is created in E:\02_work\sw\db\indicator\2005.db.
CREATE UNIQUE INDEX [HIGH_index] ON [HIGH] (date, sec_id);
[INFO] 2018-07-10 23:40:20 INDEX HIGH_index is created on TABLE HIGH of E:\02_work\sw\db\indicator\2005.db.
[INFO] 2018-07-10 23:40:20 SELECT DISTINCT(date) FROM HIGH WHERE date in ('2005-04-29', '2005-05-31', '2005-06-30', '2005-07-29', '2005-08-31', '2005-09-30', '2005-10-31', '2005-11-30', '2005-12-30')
Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2017 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.
[INFO] 2018-07-10 23:40:26 Data is inserted to HIGH successfully!
[INFO] 2018-07-10 23:40:26

[INFO] 2018-07-10 23:40:39 Data is inserted to HIGH successfully!
[INFO] 2018-07-10 23:40:39 HIGH on 2008-01-31 is updated successfully
[INFO] 2018-07-10 23:40:40 Data is inserted to HIGH successfully!
[INFO] 2018-07-10 23:40:40 HIGH on 2008-02-29 is updated successfully
[INFO] 2018-07-10 23:40:40 Data is inserted to HIGH successfully!
[INFO] 2018-07-10 23:40:40 HIGH on 2008-03-31 is updated successfully
[INFO] 2018-07-10 23:40:41 Data is inserted to HIGH successfully!
[INFO] 2018-07-10 23:40:41 HIGH on 2008-04-30 is updated successfully
[INFO] 2018-07-10 23:40:41 Data is inserted to HIGH successfully!
[INFO] 2018-07-10 23:40:41 HIGH on 2008-05-30 is updated successfully
[INFO] 2018-07-10 23:40:41 Data is inserted to HIGH successfully!
[INFO] 2018-07-10 23:40:41 HIGH on 2008-06-30 is updated successfully
[INFO] 2018-07-10 23:40:42 Data is inserted to HIGH successfully!
[INFO] 2018-07-10 23:40:42 HIGH on 2008-07-31 is updated successfully
[INFO] 2018-07-10 23:40:42 Data is inserted to H

[INFO] 2018-07-10 23:40:57 Data is inserted to HIGH successfully!
[INFO] 2018-07-10 23:40:57 HIGH on 2011-03-31 is updated successfully
[INFO] 2018-07-10 23:40:58 Data is inserted to HIGH successfully!
[INFO] 2018-07-10 23:40:58 HIGH on 2011-04-29 is updated successfully
[INFO] 2018-07-10 23:40:59 Data is inserted to HIGH successfully!
[INFO] 2018-07-10 23:40:59 HIGH on 2011-05-31 is updated successfully
[INFO] 2018-07-10 23:40:59 Data is inserted to HIGH successfully!
[INFO] 2018-07-10 23:40:59 HIGH on 2011-06-30 is updated successfully
[INFO] 2018-07-10 23:41:00 Data is inserted to HIGH successfully!
[INFO] 2018-07-10 23:41:00 HIGH on 2011-07-29 is updated successfully
[INFO] 2018-07-10 23:41:01 Data is inserted to HIGH successfully!
[INFO] 2018-07-10 23:41:01 HIGH on 2011-08-31 is updated successfully
[INFO] 2018-07-10 23:41:01 Data is inserted to HIGH successfully!
[INFO] 2018-07-10 23:41:01 HIGH on 2011-09-30 is updated successfully
[INFO] 2018-07-10 23:41:02 Data is inserted to H

[INFO] 2018-07-10 23:41:18 Data is inserted to HIGH successfully!
[INFO] 2018-07-10 23:41:18 HIGH on 2014-05-30 is updated successfully
[INFO] 2018-07-10 23:41:19 Data is inserted to HIGH successfully!
[INFO] 2018-07-10 23:41:19 HIGH on 2014-06-30 is updated successfully
[INFO] 2018-07-10 23:41:19 Data is inserted to HIGH successfully!
[INFO] 2018-07-10 23:41:19 HIGH on 2014-07-31 is updated successfully
[INFO] 2018-07-10 23:41:20 Data is inserted to HIGH successfully!
[INFO] 2018-07-10 23:41:20 HIGH on 2014-08-29 is updated successfully
[INFO] 2018-07-10 23:41:20 Data is inserted to HIGH successfully!
[INFO] 2018-07-10 23:41:20 HIGH on 2014-09-30 is updated successfully
[INFO] 2018-07-10 23:41:21 Data is inserted to HIGH successfully!
[INFO] 2018-07-10 23:41:21 HIGH on 2014-10-31 is updated successfully
[INFO] 2018-07-10 23:41:21 Data is inserted to HIGH successfully!
[INFO] 2018-07-10 23:41:21 HIGH on 2014-11-28 is updated successfully
[INFO] 2018-07-10 23:41:22 Data is inserted to H

[INFO] 2018-07-10 23:41:41 Data is inserted to HIGH successfully!
[INFO] 2018-07-10 23:41:41 HIGH on 2017-07-31 is updated successfully
[INFO] 2018-07-10 23:41:41 Data is inserted to HIGH successfully!
[INFO] 2018-07-10 23:41:41 HIGH on 2017-08-31 is updated successfully
[INFO] 2018-07-10 23:41:42 Data is inserted to HIGH successfully!
[INFO] 2018-07-10 23:41:42 HIGH on 2017-09-29 is updated successfully
[INFO] 2018-07-10 23:41:42 Data is inserted to HIGH successfully!
[INFO] 2018-07-10 23:41:42 HIGH on 2017-10-31 is updated successfully
[INFO] 2018-07-10 23:41:43 Data is inserted to HIGH successfully!
[INFO] 2018-07-10 23:41:43 HIGH on 2017-11-30 is updated successfully
[INFO] 2018-07-10 23:41:43 Data is inserted to HIGH successfully!
[INFO] 2018-07-10 23:41:43 HIGH on 2017-12-29 is updated successfully
[INFO] 2018-07-10 23:41:43 E:\02_work\sw\db\indicator\2017.db is disconnected.
[INFO] 2018-07-10 23:41:43 E:\02_work\sw\db\indicator\2018.db is connected.
CREATE TABLE [HIGH] (
    dat

...

[INFO] 2018-07-10 23:43:10 Data is inserted to IPO_LISTDAYS successfully!
[INFO] 2018-07-10 23:43:10 IPO_LISTDAYS on 2006-04-28 is updated successfully
[INFO] 2018-07-10 23:43:10 Data is inserted to IPO_LISTDAYS successfully!
[INFO] 2018-07-10 23:43:10 IPO_LISTDAYS on 2006-05-31 is updated successfully
[INFO] 2018-07-10 23:43:11 Data is inserted to IPO_LISTDAYS successfully!
[INFO] 2018-07-10 23:43:11 IPO_LISTDAYS on 2006-06-30 is updated successfully
[INFO] 2018-07-10 23:43:11 Data is inserted to IPO_LISTDAYS successfully!
[INFO] 2018-07-10 23:43:11 IPO_LISTDAYS on 2006-07-31 is updated successfully
[INFO] 2018-07-10 23:43:12 Data is inserted to IPO_LISTDAYS successfully!
[INFO] 2018-07-10 23:43:12 IPO_LISTDAYS on 2006-08-31 is updated successfully
[INFO] 2018-07-10 23:43:12 Data is inserted to IPO_LISTDAYS successfully!
[INFO] 2018-07-10 23:43:12 IPO_LISTDAYS on 2006-09-29 is updated successfully
[INFO] 2018-07-10 23:43:13 Data is inserted to IPO_LISTDAYS successfully!
[INFO] 2018-07

[INFO] 2018-07-10 23:43:24 Data is inserted to IPO_LISTDAYS successfully!
[INFO] 2018-07-10 23:43:24 IPO_LISTDAYS on 2009-02-27 is updated successfully
[INFO] 2018-07-10 23:43:25 Data is inserted to IPO_LISTDAYS successfully!
[INFO] 2018-07-10 23:43:25 IPO_LISTDAYS on 2009-03-31 is updated successfully
[INFO] 2018-07-10 23:43:25 Data is inserted to IPO_LISTDAYS successfully!
[INFO] 2018-07-10 23:43:25 IPO_LISTDAYS on 2009-04-30 is updated successfully
[INFO] 2018-07-10 23:43:26 Data is inserted to IPO_LISTDAYS successfully!
[INFO] 2018-07-10 23:43:26 IPO_LISTDAYS on 2009-05-27 is updated successfully
[INFO] 2018-07-10 23:43:26 Data is inserted to IPO_LISTDAYS successfully!
[INFO] 2018-07-10 23:43:26 IPO_LISTDAYS on 2009-06-30 is updated successfully
[INFO] 2018-07-10 23:43:27 Data is inserted to IPO_LISTDAYS successfully!
[INFO] 2018-07-10 23:43:27 IPO_LISTDAYS on 2009-07-31 is updated successfully
[INFO] 2018-07-10 23:43:27 Data is inserted to IPO_LISTDAYS successfully!
[INFO] 2018-07

[INFO] 2018-07-10 23:43:40 Data is inserted to IPO_LISTDAYS successfully!
[INFO] 2018-07-10 23:43:40 IPO_LISTDAYS on 2012-01-31 is updated successfully
[INFO] 2018-07-10 23:43:40 Data is inserted to IPO_LISTDAYS successfully!
[INFO] 2018-07-10 23:43:40 IPO_LISTDAYS on 2012-02-29 is updated successfully
[INFO] 2018-07-10 23:43:41 Data is inserted to IPO_LISTDAYS successfully!
[INFO] 2018-07-10 23:43:41 IPO_LISTDAYS on 2012-03-30 is updated successfully
[INFO] 2018-07-10 23:43:41 Data is inserted to IPO_LISTDAYS successfully!
[INFO] 2018-07-10 23:43:41 IPO_LISTDAYS on 2012-04-27 is updated successfully
[INFO] 2018-07-10 23:43:42 Data is inserted to IPO_LISTDAYS successfully!
[INFO] 2018-07-10 23:43:42 IPO_LISTDAYS on 2012-05-31 is updated successfully
[INFO] 2018-07-10 23:43:42 Data is inserted to IPO_LISTDAYS successfully!
[INFO] 2018-07-10 23:43:42 IPO_LISTDAYS on 2012-06-29 is updated successfully
[INFO] 2018-07-10 23:43:43 Data is inserted to IPO_LISTDAYS successfully!
[INFO] 2018-07

[INFO] 2018-07-10 23:43:56 INDEX IPO_LISTDAYS_index is created on TABLE IPO_LISTDAYS of E:\02_work\sw\db\indicator\2015.db.
[INFO] 2018-07-10 23:43:56 SELECT DISTINCT(date) FROM IPO_LISTDAYS WHERE date in ('2015-01-30', '2015-02-27', '2015-03-31', '2015-04-30', '2015-05-29', '2015-06-30', '2015-07-31', '2015-08-31', '2015-09-30', '2015-10-30', '2015-11-30', '2015-12-31')
[INFO] 2018-07-10 23:43:57 Data is inserted to IPO_LISTDAYS successfully!
[INFO] 2018-07-10 23:43:57 IPO_LISTDAYS on 2015-01-30 is updated successfully
[INFO] 2018-07-10 23:43:57 Data is inserted to IPO_LISTDAYS successfully!
[INFO] 2018-07-10 23:43:57 IPO_LISTDAYS on 2015-02-27 is updated successfully
[INFO] 2018-07-10 23:43:58 Data is inserted to IPO_LISTDAYS successfully!
[INFO] 2018-07-10 23:43:58 IPO_LISTDAYS on 2015-03-31 is updated successfully
[INFO] 2018-07-10 23:43:58 Data is inserted to IPO_LISTDAYS successfully!
[INFO] 2018-07-10 23:43:58 IPO_LISTDAYS on 2015-04-30 is updated successfully
[INFO] 2018-07-10 

[INFO] 2018-07-10 23:44:14 INDEX IPO_LISTDAYS_index is created on TABLE IPO_LISTDAYS of E:\02_work\sw\db\indicator\2018.db.
[INFO] 2018-07-10 23:44:14 SELECT DISTINCT(date) FROM IPO_LISTDAYS WHERE date in ('2018-01-31', '2018-02-28', '2018-03-30', '2018-04-27', '2018-05-31')
[INFO] 2018-07-10 23:44:14 Data is inserted to IPO_LISTDAYS successfully!
[INFO] 2018-07-10 23:44:14 IPO_LISTDAYS on 2018-01-31 is updated successfully
[INFO] 2018-07-10 23:44:15 Data is inserted to IPO_LISTDAYS successfully!
[INFO] 2018-07-10 23:44:15 IPO_LISTDAYS on 2018-02-28 is updated successfully
[INFO] 2018-07-10 23:44:15 Data is inserted to IPO_LISTDAYS successfully!
[INFO] 2018-07-10 23:44:15 IPO_LISTDAYS on 2018-03-30 is updated successfully
[INFO] 2018-07-10 23:44:16 Data is inserted to IPO_LISTDAYS successfully!
[INFO] 2018-07-10 23:44:16 IPO_LISTDAYS on 2018-04-27 is updated successfully
[INFO] 2018-07-10 23:44:16 Data is inserted to IPO_LISTDAYS successfully!
[INFO] 2018-07-10 23:44:16 IPO_LISTDAYS on

---

# factor

In [28]:
a = pd.DataFrame(DataAPI.schema.get_schema("factor")).T
factors = a.sort_values(['level']).index.tolist()

In [30]:
factors

['DIVIDENDYIELD2',
 'EV2_TO_EBITDA',
 'PB_LF',
 'PCF_NCF_TTM',
 'PS_TTM',
 'VAL_PE_DEDUCTED_TTM']

In [35]:
# a = pd.DataFrame(DataAPI.schema.get_schema("indicator")).T
# inds_ts = a[a.type == "时间序列"].index.tolist()
# factors4 = ["LEVERAGE","PROFIT"] # 基于其他做好的factor
# DataAPI.write.update_factors(factors=factors1,trading_days=trading_days , override=False, log=False)
# DataAPI.write.update_factors(factors=factors2,trading_days=trading_days , override=False, log=False)
# DataAPI.write.update_factors(factors=factors3,trading_days=trading_days , override=False, log=False)
DataAPI.write.update_factors(factors=factors,trading_days=trading_days , override=False, log=False)

[INFO] 2018-06-28 14:14:56 Updating factor DIVIDENDYIELD2
[INFO] 2018-06-28 14:14:56 schema updated: DIVIDENDYIELD2
[INFO] 2018-06-28 14:14:56 Updating factor EV2_TO_EBITDA
[INFO] 2018-06-28 14:14:56 schema updated: EV2_TO_EBITDA
[INFO] 2018-06-28 14:14:56 Updating factor PB_LF
[INFO] 2018-06-28 14:14:56 schema updated: PB_LF
[ERROR] 2018-06-28 14:14:57 Unrecognized factor: PCF_NCF_TTM
[INFO] 2018-06-28 14:14:57 ------------------------------------------
[INFO] 2018-06-28 14:14:57 Updating factor PS_TTM
[INFO] 2018-06-28 14:14:57 TABLE PS_TTM is created in E:\02_work\sw\db\factor\2005.db.
[INFO] 2018-06-28 14:14:57 INDEX PS_TTM_index is created on TABLE PS_TTM of E:\02_work\sw\db\factor\2005.db.
[INFO] 2018-06-28 14:14:59 TABLE PS_TTM is created in E:\02_work\sw\db\factor\2006.db.
[INFO] 2018-06-28 14:14:59 INDEX PS_TTM_index is created on TABLE PS_TTM of E:\02_work\sw\db\factor\2006.db.
[INFO] 2018-06-28 14:15:02 TABLE PS_TTM is created in E:\02_work\sw\db\factor\2007.db.
[INFO] 2018-

In [5]:
DataAPI.write.update_factors(factors=['GROWTH'],trading_days=trading_days, override=True, log=False)

[ERROR] 2018-06-28 13:18:32 Unrecognized factor: GROWTH
[INFO] 2018-06-28 13:18:32 ------------------------------------------


---

# factor_return

In [27]:
# factors = ["SIZE", 'VALUE', 'BETA', 'VOL', 'ROE', 'MOM', 'LIQ'] # 为空表示更新所有存在的factors
factors = []
DataAPI.write.update_factors_return(['GROWTH'], trading_days)

[INFO] 2018-03-14 16:13:50 Updating factor_return GROWTH...
[INFO] 2018-03-14 16:14:57 存量更新：更新当前已经有的记录
[INFO] 2018-03-14 16:14:57 schema updated: GROWTH
[INFO] 2018-03-14 16:14:57 传入日期数：1015  已经存在个数：1  实际写入次数：1014
[INFO] 2018-03-14 16:14:57 factor_return GROWTH is updated.
[INFO] 2018-03-14 16:14:57 ------------------------------------------
